License check

In [1]:
!pip install gurobipy
import gurobipy as gp
from google.colab import userdata

try:
    # Create an environment with your WLS license
    # Access secrets manager for WLSACCESSID and WLSSECRET
    params = {
        "WLSACCESSID": "b92cfff8-0279-4652-9cb7-3e6bcdb967cd",
        "WLSSECRET": "e6220bd1-2657-40f8-ba01-6cebcb97db7fdesearc",
        "LICENSEID": 2726201,
    }
    env = gp.Env(params=params)

    print("Gurobi License Information:")
    print(f"  WLS Access ID: {env.getParam('WLSACCESSID')}")
    print(f"  WLS Secret: {env.getParam('WLSSECRET')}")
    print(f"  License ID: {env.getParam('LICENSEID')}")
    # Accessing LicenseStatus directly might not always be possible or reliable
    # print(f"  License Status: {env.getParam('LicenseStatus')}")

    # You might need to perform a simple operation to get more status details
    # For example, creating a simple model and optimizing it briefly
    # model = gp.Model(env=env)
    # model.addVar(name="dummy")
    # model.setObjective(1.0, GRB.MINIMIZE)
    # model.optimize()
    # print(f"  Gurobi Status (after dummy optimization): {model.Status}")
    # del model # Clean up the dummy model


except gp.GurobiError as e:
    print(f"Error creating Gurobi environment or retrieving license information: {e}")
    print("Please ensure your WLS license details (WLSACCESSID, WLSSECRET, LICENSEID) are correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 70.3 MB/s eta 0:00:00
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2726201
Error creating Gurobi environment or retrieving license information: Unauthorized access
Please ensure your WLS license details (WLSACCESSID, WLSSECRET, LICENSEID) are correct.


# Cleaning

In [2]:
# =========================================
# Cell A — install packages (gurobipy)
# =========================================
import sys, subprocess, importlib
def _pip_install(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
_pip_install("gurobipy")
print("✓ gurobipy installed/available")

# =========================================
# Cell B — mount Google Drive & set paths
# =========================================
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted")
except Exception as e:
    print("Not in Colab or Drive mount failed:", e)

import os
import pandas as pd
import numpy as np

BASE_PATH   = '/content/drive/MyDrive/IEORE4004 Optimization Models and Methods/'
DATA_PATH   = os.path.join(BASE_PATH, 'IEOR4404_Proj1/data/')
OUTPUT_PATH = os.path.join(BASE_PATH, 'IEOR4404_Proj1/cleaned_data/')
os.makedirs(OUTPUT_PATH, exist_ok=True)
print("DATA_PATH:", DATA_PATH)
print("OUTPUT_PATH:", OUTPUT_PATH)
print("DATA exists:", os.path.exists(DATA_PATH))

# =========================================
# Cell C — load csvs
# =========================================
def _read_csv(path, **kwargs):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")
    return pd.read_csv(path, **kwargs)

df_facilities  = _read_csv(os.path.join(DATA_PATH, 'child_care_regulated.csv'))
df_population  = _read_csv(os.path.join(DATA_PATH, 'population.csv'))
df_income      = _read_csv(os.path.join(DATA_PATH, 'avg_individual_income.csv'))
df_employment  = _read_csv(os.path.join(DATA_PATH, 'employment_rate.csv'))
df_locations   = _read_csv(os.path.join(DATA_PATH, 'potential_locations.csv'))

print("Loaded shapes:",
      "fac", df_facilities.shape,
      "pop", df_population.shape,
      "inc", df_income.shape,
      "emp", df_employment.shape,
      "loc", df_locations.shape)

# =========================================
# Cell D — impute facility coordinates
# =========================================
zip_col_loc = 'zip_code' if 'zip_code' in df_locations.columns else 'zipcode'
zip_col_fac = 'zip_code' if 'zip_code' in df_facilities.columns else 'zipcode'

_zip = df_locations.dropna(subset=['latitude','longitude']).copy()
_zip[zip_col_loc] = _zip[zip_col_loc].astype(str)

centroids = (
    _zip.groupby(zip_col_loc, as_index=False)[['latitude','longitude']]
        .mean()
        .rename(columns={'latitude':'lat_centroid','longitude':'lon_centroid'})
)

_fac = df_facilities.copy()
_fac[zip_col_fac] = _fac[zip_col_fac].astype(str)
_fac = _fac.merge(centroids, left_on=zip_col_fac, right_on=zip_col_loc, how='left')

fac_means = (
    _fac.dropna(subset=['latitude','longitude'])
        .groupby(zip_col_fac)[['latitude','longitude']].mean()
        .rename(columns={'latitude':'lat_zip_fac_mean','longitude':'lon_zip_fac_mean'})
)
_fac = _fac.join(fac_means, on=zip_col_fac)

global_lat = _zip['latitude'].mean() if not _zip.empty else df_facilities['latitude'].mean()
global_lon = _zip['longitude'].mean() if not _zip.empty else df_facilities['longitude'].mean()

_fac['latitude']  = _fac['latitude'].fillna(_fac.get('lat_centroid')).fillna(_fac.get('lat_zip_fac_mean')).fillna(global_lat)
_fac['longitude'] = _fac['longitude'].fillna(_fac.get('lon_centroid')).fillna(_fac.get('lon_zip_fac_mean')).fillna(global_lon)

df_facilities[['latitude','longitude']] = _fac[['latitude','longitude']]
print("✓ imputed facility coordinates; nulls:",
      df_facilities[['latitude','longitude']].isna().sum().to_dict())

# =========================================
# Cell E — standardize columns
# =========================================
df_facilities.rename(columns={'zip_code':'zipcode', 'total_capacity':'capacity'}, inplace=True)
df_income.rename(columns={'ZIP code':'zipcode', 'average income':'avg_income'}, inplace=True)
df_employment.rename(columns={'employment rate':'employment_rate'}, inplace=True)
print("✓ standardized column names")

# normalize zipcode dtype early (str + zfill(5))
for df_ in [df_facilities, df_population, df_income, df_employment, df_locations]:
    if 'zipcode' in df_.columns:
        df_['zipcode'] = df_['zipcode'].astype(str).str.zfill(5)
print("✓ zipcodes normalized to str zfill(5)")

# =========================================
# Cell F — compute H025/H512 with residual allocation (FIX)
# =========================================
fac = df_facilities.copy()
fac['h025_base'] = fac[['infant_capacity','toddler_capacity','preschool_capacity']].fillna(0).sum(axis=1)
fac['h512_base'] = fac['school_age_capacity'].fillna(0)
fac['residual']  = (fac['capacity'] - (fac['h025_base'] + fac['h512_base'])).clip(lower=0)

# zip-level share_0_5 from population (0–5 / (0–5 + 0.795778952*(5–9 + 10–14)))
# https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-detail.html

pop_share = (
    df_population.assign(
        share_0_5 = df_population['-5'] /
                    (df_population['-5'] + 0.795778952*(df_population['5-9'] + df_population['10-14']))
    )[["zipcode","share_0_5"]]
)
pop_share['share_0_5'] = pop_share['share_0_5'].replace([np.inf, -np.inf], np.nan)
pop_share['share_0_5'] = pop_share['share_0_5'].fillna(pop_share['share_0_5'].mean())

fac = fac.merge(pop_share, on='zipcode', how='left')
fac['H025_facility'] = fac['h025_base'] + fac['residual'] * fac['share_0_5'].fillna(0.38)
fac['H512_facility'] = fac['h512_base'] + fac['residual'] * (1 - fac['share_0_5'].fillna(0.38))

zip_capacity_by_age = (
    fac.groupby('zipcode', as_index=False)
       .agg(H025_zip=('H025_facility','sum'),
            H512_zip=('H512_facility','sum'),
            total_capacity=('capacity','sum'),
            facility_count=('facility_id','count'))
)
zip_out = os.path.join(OUTPUT_PATH, 'zip_capacity_by_age.csv')
zip_capacity_by_age.to_csv(zip_out, index=False)
print("✓ computed & saved H025/H512:", zip_out)

# =========================================
# Cell G — save enriched facilities now (for downstream)
# =========================================
fac_out = os.path.join(OUTPUT_PATH, 'facilities_clean.csv')
df_facilities.to_csv(fac_out, index=False)
print("✓ wrote", fac_out)

# =========================================
# Cell H — reconcile totals (dtype-safe) and write zipcode_master_reconciled.csv (FIX)
# =========================================
fac_zip = df_facilities.copy()
fac_zip['zipcode'] = fac_zip['zipcode'].astype(str).str.zfill(5)

cap_by_zip = (
    fac_zip.groupby('zipcode', as_index=False)['capacity']
           .sum()
           .rename(columns={'capacity': 'capacity_from_fac'})
)

# try load an existing master (dtype=str); else start from cap_by_zip
candidates = [
    os.path.join(OUTPUT_PATH, 'zipcode_master.csv'),
    os.path.join(OUTPUT_PATH, 'zipcode_master_reconciled.csv'),
    os.path.join(DATA_PATH,   'zipcode_master.csv')
]
master_path = next((p for p in candidates if os.path.exists(p)), None)
if master_path:
    zipcode_master = pd.read_csv(master_path, dtype={'zipcode': str})
else:
    zipcode_master = cap_by_zip[['zipcode']].copy()

zipcode_master['zipcode'] = zipcode_master['zipcode'].astype(str).str.zfill(5)
cap_by_zip['zipcode']      = cap_by_zip['zipcode'].astype(str).str.zfill(5)

zipcode_master = zipcode_master.merge(cap_by_zip, on='zipcode', how='left')

if 'total_existing_slots' in zipcode_master.columns:
    zipcode_master['gap_vs_old'] = zipcode_master['total_existing_slots'] - zipcode_master['capacity_from_fac']

zipcode_master['total_existing_slots'] = zipcode_master['capacity_from_fac'].fillna(0)

# add current coverage ratio if population is present
if 'pop_0_12' in zipcode_master.columns:
    zipcode_master['current_coverage_ratio'] = zipcode_master['total_existing_slots'] / zipcode_master['pop_0_12'].replace(0, np.nan)

outp_recon = os.path.join(OUTPUT_PATH, 'zipcode_master_reconciled.csv')
zipcode_master.to_csv(outp_recon, index=False)
print("✓ wrote", outp_recon)

# =========================================
# Cell I — compute population groups (0–5, 5–12, 0–12)
# =========================================
df_population_clean = df_population[['zipcode','-5','5-9','10-14']].copy()
df_population_clean['pop_0_5']  = df_population_clean['-5']
df_population_clean['pop_5_12'] = 0.7957 * (df_population_clean['5-9'] + df_population_clean['10-14']) # from nys census data 2020-2024
df_population_clean['pop_0_12'] = df_population_clean['pop_0_5'] + df_population_clean['pop_5_12']
df_population_clean['pop_5_12'] = df_population_clean['pop_5_12'].round()
df_population_clean['pop_0_12'] = df_population_clean['pop_0_12'].round()
df_population_clean = df_population_clean[['zipcode','pop_0_5','pop_5_12','pop_0_12']]

print("✓ built population aggregates")
print("Total 0–12:", int(df_population_clean['pop_0_12'].sum()))

# =========================================
# Cell J — aggregate facilities by zip
# =========================================
facilities_agg = (
    df_facilities.groupby('zipcode', as_index=False)
                 .agg(total_existing_slots=('capacity','sum'),
                      num_existing_facilities=('facility_id','count'))
)
print("✓ facilities_agg:", facilities_agg.shape)

# =========================================
# Cell K — master zipcode list
# =========================================
all_zipcodes = set()
for df_ in [df_facilities, df_population_clean, df_income, df_employment, df_locations]:
    if 'zipcode' in df_.columns:
        all_zipcodes.update(df_['zipcode'].astype(str).str.zfill(5).unique())
df_master = pd.DataFrame({'zipcode': sorted(list(all_zipcodes))})
print("✓ master zipcode list:", len(df_master))

# =========================================
# Cell L — merge all data (and keep H025/H512) (FIX)
# =========================================
df_master['zipcode']             = df_master['zipcode'].astype(str).str.zfill(5)
df_population_clean['zipcode']   = df_population_clean['zipcode'].astype(str).str.zfill(5)
df_income['zipcode']             = df_income['zipcode'].astype(str).str.zfill(5)
df_employment['zipcode']         = df_employment['zipcode'].astype(str).str.zfill(5)
facilities_agg['zipcode']        = facilities_agg['zipcode'].astype(str).str.zfill(5)
zip_capacity_by_age['zipcode']   = zip_capacity_by_age['zipcode'].astype(str).str.zfill(5)

df_master = df_master.merge(df_population_clean, on='zipcode', how='left')
df_master = df_master.merge(df_income[['zipcode','avg_income']], on='zipcode', how='left')
df_master = df_master.merge(df_employment[['zipcode','employment_rate']], on='zipcode', how='left')
df_master = df_master.merge(facilities_agg, on='zipcode', how='left')
df_master['total_existing_slots']   = df_master['total_existing_slots'].fillna(0)
df_master['num_existing_facilities']= df_master['num_existing_facilities'].fillna(0)

# merge 0–5 / 5–12 baselines
df_master = df_master.merge(zip_capacity_by_age[['zipcode','H025_zip','H512_zip']], on='zipcode', how='left')
df_master[['H025_zip','H512_zip']] = df_master[['H025_zip','H512_zip']].fillna(0)

print("✓ merged master with population/income/employment/facilities + H025/H512")
print("Missing values:\n", df_master.isnull().sum())

# =========================================
# Cell M — identify data quality issues
# =========================================
missing_pop        = df_master[df_master['pop_0_12'].isnull()]
missing_income     = df_master[df_master['avg_income'].isnull()]
missing_employment = df_master[df_master['employment_rate'].isnull()]
print(f"⚠️ Missing population: {len(missing_pop)} | income: {len(missing_income)} | employment: {len(missing_employment)}")

# =========================================
# Cell N — drop rows missing critical fields
# =========================================
before = len(df_master)
df_clean = df_master.dropna(subset=['pop_0_12','avg_income','employment_rate']).copy()
after = len(df_clean)
print(f"✓ cleaned zipcodes: {before} → {after} (dropped {before-after})")
assert df_clean[['pop_0_12','pop_0_5','pop_5_12','avg_income','employment_rate']].isnull().sum().sum() == 0

# =========================================
# Cell O — derived fields (Problem 1: budgeting) (no 0–5 constraint enforced here)
# =========================================
df_clean['high_demand'] = ((df_clean['employment_rate'] >= 0.60) | (df_clean['avg_income'] <= 60000)).astype(int)
df_clean['desert_threshold'] = np.where(
    df_clean['high_demand'] == 1,
    0.5 * df_clean['pop_0_12'],
    0.33 * df_clean['pop_0_12']
)

df_clean['is_desert'] = (df_clean['total_existing_slots'] <= df_clean['desert_threshold']).astype(int)
df_clean['current_coverage_ratio'] = df_clean['total_existing_slots'] / df_clean['pop_0_12'].replace(0, np.nan)
print("✓ derived fields added")

# =========================================
# Cell P — summary
# =========================================
print("="*60)
print("CLEANED DATA SUMMARY")
print("="*60)
print(f"Zipcodes: {len(df_clean)} | Deserts: {df_clean['is_desert'].sum()} ({100*df_clean['is_desert'].mean():.1f}%)")
print(f"High-demand: {df_clean['high_demand'].sum()} ({100*df_clean['high_demand'].mean():.1f}%)")
print(f"Zero-facility zips: {(df_clean['total_existing_slots'] == 0).sum()}")
print(f"Total children 0–12: {int(df_clean['pop_0_12'].sum()):,}")
print(f"Total children 0–5 : {int(df_clean['pop_0_5'].sum()):,}")
print(f"Total existing slots: {int(df_clean['total_existing_slots'].sum()):,}")
print(f"Overall coverage ratio: {df_clean['total_existing_slots'].sum() / df_clean['pop_0_12'].sum():.3f}")

# =========================================
# Cell Q — clean facilities for downstream optimization
# =========================================
valid_zipcodes = set(df_clean['zipcode'].unique())
df_facilities_clean = df_facilities[df_facilities['zipcode'].isin(valid_zipcodes)].copy()
df_facilities_clean = df_facilities_clean.dropna(subset=['capacity','latitude','longitude']).copy()
df_facilities_clean = df_facilities_clean[['facility_id','zipcode','capacity','latitude','longitude']]
print("✓ facilities cleaned:", df_facilities_clean.shape)

# =========================================
# Cell R — clean potential locations
# =========================================
df_locations_clean = df_locations[df_locations['zipcode'].isin(valid_zipcodes)].copy()
df_locations_clean = df_locations_clean.reset_index(drop=True)
df_locations_clean['location_id'] = range(len(df_locations_clean))
print("✓ locations cleaned:", df_locations_clean.shape)

# =========================================
# Cell S — save all cleaned outputs
# =========================================
df_clean.to_csv(os.path.join(OUTPUT_PATH, 'zipcode_master.csv'), index=False)
df_facilities_clean.to_csv(os.path.join(OUTPUT_PATH, 'facilities_clean.csv'), index=False)
df_locations_clean.to_csv(os.path.join(OUTPUT_PATH, 'locations_clean.csv'), index=False)
zip_capacity_by_age.to_csv(os.path.join(OUTPUT_PATH, 'zip_capacity_by_age.csv'), index=False)
print("✓ saved zipcode_master.csv / facilities_clean.csv / locations_clean.csv / zip_capacity_by_age.csv")

# also (re)write reconciled using the authoritative facility sum already computed above
zipcode_master_reconciled = df_clean.copy()
# ensure reconciled carries the authoritative totals (capacity_from_fac)
cap_from_fac = cap_by_zip.rename(columns={'capacity_from_fac':'total_existing_slots'})
zipcode_master_reconciled = zipcode_master_reconciled.drop(columns=['total_existing_slots'], errors='ignore')
zipcode_master_reconciled = zipcode_master_reconciled.merge(cap_from_fac, on='zipcode', how='left')
zipcode_master_reconciled['total_existing_slots'] = zipcode_master_reconciled['total_existing_slots'].fillna(0)
zipcode_master_reconciled.to_csv(os.path.join(OUTPUT_PATH, 'zipcode_master_reconciled.csv'), index=False)
print("✓ rewrote zipcode_master_reconciled.csv")

# =========================================
# Cell T — quick validation report (non-fatal)
# =========================================
print("\nVALIDATION REPORT")
print("="*60)
crit_nulls = df_clean[['zipcode','pop_0_12','pop_0_5','pop_5_12','avg_income','employment_rate']].isnull().sum().sum()
print("Critical nulls:", crit_nulls)
desert_logic_high = ((df_clean[df_clean['high_demand']==1]['is_desert'] ==
                      (df_clean[df_clean['high_demand']==1]['total_existing_slots'] <= 0.5*df_clean[df_clean['high_demand']==1]['pop_0_12'])).all())
desert_logic_norm = ((df_clean[df_clean['high_demand']==0]['is_desert'] ==
                      (df_clean[df_clean['high_demand']==0]['total_existing_slots'] <= 0.33*df_clean[df_clean['high_demand']==0]['pop_0_12'])).all())
print("Desert logic (high-demand):", desert_logic_high)
print("Desert logic (normal)    :", desert_logic_norm)
pop_math = (df_clean['pop_0_5'] + df_clean['pop_5_12'] == df_clean['pop_0_12']).all()
print("Population math correct  :", pop_math)
print("="*60)
print("DONE")

# Assign cleaned dataframes to names expected by downstream cells
pl = df_locations_clean.copy()
zipdf = df_clean.copy() # Assign df_clean to zipdf
print("✓ defined 'pl' and 'zipdf' dataframes from cleaned data")

✓ gurobipy installed/available
Mounted at /content/drive
Google Drive mounted
DATA_PATH: /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/data/
OUTPUT_PATH: /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/
DATA exists: True
Loaded shapes: fac (15604, 15) pop (1646, 20) inc (1534, 2) emp (1375, 2) loc (215400, 3)
✓ imputed facility coordinates; nulls: {'latitude': 0, 'longitude': 0}
✓ standardized column names
✓ zipcodes normalized to str zfill(5)
✓ computed & saved H025/H512: /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/zip_capacity_by_age.csv
✓ wrote /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/facilities_clean.csv
✓ wrote /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/zipcode_master_reconciled.csv
✓ built population aggregates
Total 0–12: 2812163
✓ facilities_agg: (1604

In [3]:
print("Total facilities (raw, unique IDs):", df_facilities['facility_id'].nunique())
print("Total facilities (raw, rows):", len(df_facilities))

print("Total facilities (cleaned):", len(df_facilities_clean))
print("Sum of per-ZIP counts (facilities_agg):", int(facilities_agg['num_existing_facilities'].sum()))
print("Sum of per-ZIP counts (zip_capacity_by_age):", int(zip_capacity_by_age['facility_count'].sum()))


Total facilities (raw, unique IDs): 15604
Total facilities (raw, rows): 15604
Total facilities (cleaned): 14435
Sum of per-ZIP counts (facilities_agg): 15604
Sum of per-ZIP counts (zip_capacity_by_age): 15604


# Part 1

## Optimiation
fixed the 200 per existing spot, and 200000 baseline, i think

In [4]:
# ===========================
# Step 1 — install & imports
# ===========================
import sys, subprocess, importlib, os

def _pip_install(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for _pkg in ("gurobipy","pandas","numpy"):
    _pip_install(_pkg)

import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# ================================
# Step 2 — mount Drive & set paths
# ================================
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    print("✓ Google Drive mounted.")
except Exception as e:
    print("Not in Colab or Drive mount failed:", e)

BASE_PATH   = "/content/drive/MyDrive/IEORE4004 Optimization Models and Methods/"
DATA_PATH   = os.path.join(BASE_PATH, "IEOR4404_Proj1/data/")
OUTPUT_PATH = os.path.join(BASE_PATH, "IEOR4404_Proj1/cleaned_data/")
os.makedirs(OUTPUT_PATH, exist_ok=True)
print("DATA_PATH:", DATA_PATH)
print("OUTPUT_PATH:", OUTPUT_PATH)

# ===========================================
# Step 3 — load cleaned inputs (dtype-safe zips)
# ===========================================
def _read_csv_safe(path, dtype_zip=True):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")
    if dtype_zip:
        return pd.read_csv(path, dtype={"zipcode": str})
    return pd.read_csv(path)

master_path_csv = os.path.join(OUTPUT_PATH, "zipcode_master.csv")
fac_path_csv    = os.path.join(OUTPUT_PATH, "facilities_clean.csv")
loc_path_csv    = os.path.join(OUTPUT_PATH, "locations_clean.csv")
zipage_path_csv = os.path.join(OUTPUT_PATH, "zip_capacity_by_age.csv")

master = _read_csv_safe(master_path_csv)
fac    = _read_csv_safe(fac_path_csv)
locs   = _read_csv_safe(loc_path_csv)
zipage = _read_csv_safe(zipage_path_csv)

# normalize zipcode format
for df_ in (master, fac, locs, zipage):
    if "zipcode" in df_.columns:
        df_["zipcode"] = df_["zipcode"].astype(str).str.zfill(5)

# quick guards: required columns
req_master = {"zipcode","pop_0_12","total_existing_slots","desert_threshold"}
missing_master_cols = req_master - set(master.columns)
if missing_master_cols:
    raise ValueError(f"zipcode_master.csv missing columns: {missing_master_cols}")

req_fac = {"facility_id","zipcode","capacity"}
if not req_fac.issubset(set(fac.columns)):
    raise ValueError(f"facilities_clean.csv missing columns: {req_fac - set(fac.columns)}")

# synthesize location_id if missing
if "location_id" not in locs.columns:
    locs = locs.reset_index(drop=True)
    locs["location_id"] = np.arange(len(locs))

req_locs = {"location_id","zipcode"}
if not req_locs.issubset(set(locs.columns)):
    raise ValueError(f"locations_clean.csv missing columns: {req_locs - set(locs.columns)}")

print("✓ Loaded cleaned inputs.",
      "\n master:", master.shape,
      "\n fac   :", fac.shape,
      "\n locs  :", locs.shape,
      "\n zipage:", zipage.shape)

# ======================================================
# Step 4 — create Gurobi WLS environment
# ======================================================
params = {
    "WLSACCESSID": "b92cfff8-0279-4652-9cb7-3e6bcdb967cd",
    "WLSSECRET":   "e6220bd1-2657-40f8-ba01-6cebcb97db7f",
    "LICENSEID":   2726201,
}
env = gp.Env(params=params)
print("✓ Gurobi WLS environment created.")

# =================================================================
# Step 5 — sets/parameters (Problem 1: C1 + C2)
# =================================================================
# Sets
Z = sorted(master["zipcode"].unique().tolist())
F = fac["facility_id"].astype(int).tolist()
P = locs["location_id"].astype(int).tolist()
TYPES = ["S","M","L"]

# Index maps
z_of_f = dict(zip(fac["facility_id"].astype(int), fac["zipcode"]))
z_of_p = dict(zip(locs["location_id"].astype(int), locs["zipcode"]))

# Parameters
K_total = {"S":100, "M":200, "L":400}
K_0_5   = {"S": 50, "M":100, "L":200}
c_build = {"S": 65000, "M": 95000, "L":115000}
c_equip = 100.0  # per 0–5 slot equipment cost

# corrected expansion charging:
# if expand by >=100% at facility f (x_f >= current capacity nf), pay a fixed cost:
# 20000 + 200 * nf, once per such facility (not per added slot)
baseline_cost = 20000.0

# Existing facility capacity & P1 bounds (add <=120% of current, total cap <=500)
Cnow_f = dict(zip(fac["facility_id"].astype(int), fac["capacity"]))
xmax_f = {}
for f_id in F:
    cnow = float(Cnow_f.get(f_id, 0.0))
    add_cap_120 = 1.2 * cnow
    room_to_500 = max(0.0, 500.0 - cnow)
    xmax_f[f_id] = min(add_cap_120, room_to_500)

# ZIP-level totals for 0–12 (C1 target in slots)
Cnow_tot     = dict(zip(master["zipcode"], master["total_existing_slots"]))
Target_slots = dict(zip(master["zipcode"], master["desert_threshold"]))

# ZIP-level 0–5 data (C2 target)
if "pop_0_5" in master.columns:
    N0_5_series = master.set_index("zipcode")["pop_0_5"].astype(float)
elif "pop_0_5" in zipage.columns:
    N0_5_series = zipage.groupby("zipcode")["pop_0_5"].sum().astype(float)
else:
    raise ValueError("Need pop_0_5 in master or zip_capacity_by_age.csv for C2.")

if "H025_zip" in master.columns:
    Cnow_05_series = master.set_index("zipcode")["H025_zip"].astype(float)
elif "H025_zip" in zipage.columns:
    Cnow_05_series = zipage.groupby("zipcode")["H025_zip"].sum().astype(float)
else:
    raise ValueError("Need H025_zip in master or zip_capacity_by_age.csv for C2.")

# Align to full ZIP index Z
N0_5_series = N0_5_series.reindex(Z).fillna(0.0)
Cnow_05_series = Cnow_05_series.reindex(Z).fillna(0.0)

# Convenience: entities in each ZIP
Fz = {z: [f_id for f_id in F if z_of_f[f_id] == z] for z in Z}
Pz = {z: [p_id for p_id in P if z_of_p[p_id] == z] for z in Z}

# Deficits and needy ZIPs
tau_05 = 2.0/3.0
deficit12_by_zip = {z: max(0.0, float(Target_slots.get(z,0.0)) - float(Cnow_tot.get(z,0.0))) for z in Z}
deficit05_by_zip = {z: max(0.0, float(tau_05 * N0_5_series.loc[z]) - float(Cnow_05_series.loc[z])) for z in Z}

needy_12 = [z for z in Z if deficit12_by_zip[z] > 1e-6]
needy_05 = [z for z in Z if deficit05_by_zip[z] > 1e-6]
need_union = sorted(set(needy_12) | set(needy_05))

# Filter candidate sites to union of needy ZIPs
P = [p for p in P if z_of_p[p] in need_union]
Pz = {z: [p for p in P if z_of_p[p] == z] for z in Z}
print(f"ZIPs with need — C1: {len(needy_12)}, C2: {len(needy_05)}, union: {len(need_union)}; kept sites: {len(P)}")

# Optional: cap sites per ZIP
MAX_SITES_PER_ZIP = 20
if MAX_SITES_PER_ZIP is not None:
    keepP = []
    for z in need_union:
        keepP.extend(Pz[z][:MAX_SITES_PER_ZIP])
    P = sorted(set(keepP))
    Pz = {z: [p for p in P if z_of_p[p] == z] for z in Z}
    print(f"Sites after per-ZIP cap ({MAX_SITES_PER_ZIP}): {len(P)}")

# Quick feasibility upper bounds per ZIP
def max_supply_12(z):
    return sum(xmax_f.get(f,0.0) for f in Fz[z]) + len(Pz[z]) * K_total['L']
def max_supply_05(z):
    return sum(xmax_f.get(f,0.0) for f in Fz[z]) + len(Pz[z]) * K_0_5['L']

bad_12 = [z for z in Z if deficit12_by_zip[z] > max_supply_12(z) + 1e-6]
bad_05 = [z for z in Z if deficit05_by_zip[z] > max_supply_05(z) + 1e-6]
print("C1 impossible ZIPs:", len(bad_12), bad_12[:10])
print("C2 impossible ZIPs:", len(bad_05), bad_05[:10])

needy_12 = [z for z in needy_12 if z not in bad_12]
needy_05 = [z for z in needy_05 if z not in bad_05]

# ==========================================
# Step 6 — build model (Problem 1: C1 + C2)
# ==========================================
m = gp.Model("P1_Idealistic_Budgeting", env=env)
eps = 1e-6

# Decision vars
x = m.addVars(F, name="x", lb=0.0)                      # added total slots at existing fac
a = m.addVars(F, name="a", lb=0.0)                      # 0–5 portion within x
y = m.addVars(P, TYPES, vtype=GRB.BINARY, name="y")     # build type at site
s = m.addVars(P, TYPES, lb=0.0, name="s")               # 0–5 at new site/type

# indicator for >=100% expansion at facility f (x_f >= current capacity)
b100 = m.addVars(F, vtype=GRB.BINARY, name="b100")

# Objective (min cost)
obj_build = gp.quicksum(c_build[t] * y[p,t] for p in P for t in TYPES)

# corrected expansion cost: pay once per facility if x_f >= current capacity
obj_exp_fixed = gp.quicksum(
    b100[f] * (baseline_cost + 200.0 * float(Cnow_f.get(f,0.0)))
    for f in F
)

obj_equip = 100.0 * (gp.quicksum(s[p,t] for p in P for t in TYPES) + gp.quicksum(a[f] for f in F))
m.setObjective(obj_build + obj_exp_fixed + obj_equip, GRB.MINIMIZE)

# Linking / bounds
for f in F:
    m.addConstr(x[f] <= xmax_f[f], name=f"bound_x_{f}")
    m.addConstr(a[f] <= x[f],      name=f"bound_a_le_x_{f}")

    cnow = float(Cnow_f.get(f, 0.0))
    if cnow > 0.0:
        # if b100=0 then x <= cnow - eps; if x >= cnow then must set b100=1
        m.addConstr(x[f] <= (cnow - eps) + xmax_f[f]*b100[f], name=f"trigger_up_{f}")
        # if b100=1 then x >= cnow
        m.addConstr(x[f] >= cnow * b100[f], name=f"trigger_lo_{f}")
    else:
        # no current capacity -> do not charge the step cost
        m.addConstr(b100[f] == 0, name=f"trigger_zero_{f}")

for p in P:
    for t in TYPES:
        m.addConstr(s[p,t] <= K_0_5[t] * y[p,t], name=f"bound_s_le_K05y_{p}_{t}")

# at most one facility type per site
for p in P:
    m.addConstr(gp.quicksum(y[p,t] for t in TYPES) <= 1, name=f"C_one_type_per_site_{p}")

# Coverage — C1 (0–12 total)
for z in needy_12:
    lhs_12 = gp.quicksum(K_total[t] * y[p,t] for p in Pz[z] for t in TYPES) + gp.quicksum(x[f] for f in Fz[z])
    rhs_12 = deficit12_by_zip[z]
    m.addConstr(lhs_12 >= rhs_12, name=f"C1_cover12_{z}")

# Coverage — C2 (0–5 share)
for z in needy_05:
    lhs_05 = gp.quicksum(s[p,t] for p in Pz[z] for t in TYPES) + gp.quicksum(a[f] for f in Fz[z])
    rhs_05 = deficit05_by_zip[z]
    m.addConstr(lhs_05 >= rhs_05, name=f"C2_cover05_{z}")

m.update()
print(f"Model size: {m.NumVars} vars, {m.NumConstrs} rows")

# ======================================================
# Step 7 — solve
# ======================================================
m.Params.MIPGap    = 1e-3
m.Params.TimeLimit = 600
m.optimize()

# If infeasible, write IIS to OUTPUT_PATH
status_map = {
    GRB.OPTIMAL: "OPTIMAL",
    GRB.TIME_LIMIT: "TIME_LIMIT",
    GRB.SUBOPTIMAL: "SUBOPTIMAL",
    GRB.INTERRUPTED: "INTERRUPTED",
    GRB.INFEASIBLE: "INFEASIBLE",
    GRB.INF_OR_UNBD: "INF_OR_UNBD",
    GRB.UNBOUNDED: "UNBOUNDED",
}
status_txt = status_map.get(m.Status, f"STATUS_{m.Status}")

if m.Status == GRB.INFEASIBLE:
    try:
        m.computeIIS()
        iis_path = os.path.join(OUTPUT_PATH, "p1_budgeting.iis")
        m.write(iis_path)
        print("IIS written to:", iis_path)
    except Exception as _:
        pass

obj_val = m.objVal if m.SolCount > 0 else None

# ===============================================
# Step 8 — extract solution + coverage validation
# ===============================================
# expansions
exp_rows = []
if m.SolCount > 0:
    for f_id in F:
        xv = x[f_id].X
        av = a[f_id].X
        bv = int(round(b100[f_id].X))
        if xv > 1e-6:
            fixed_cost = (baseline_cost + 200.0 * float(Cnow_f.get(f_id,0.0))) if bv == 1 else 0.0
            exp_rows.append({
                "facility_id": f_id,
                "zipcode": z_of_f[f_id],
                "expansion_slots": float(xv),
                "expansion_0_5": float(av),
                "trigger_ge_100pct": bv,
                "cost_expansion_fixed": fixed_cost,
                "cost_equip_0_5_exp": float(av) * c_equip,
            })
exp_df = pd.DataFrame(exp_rows)

# new builds
new_rows = []
if m.SolCount > 0:
    for p in P:
        for t in TYPES:
            yv = y[p,t].X
            sv = s[p,t].X
            if yv > 0.5:
                new_rows.append({
                    "location_id": p,
                    "zipcode": z_of_p[p],
                    "type": t,
                    "built": int(round(yv)),
                    "added_slots_total": K_total[t] * int(round(yv)),
                    "added_slots_0_5_designated": float(sv),
                    "cost_build": c_build[t] * int(round(yv)),
                    "cost_equip_0_5_new": float(sv) * c_equip,
                })
new_df = pd.DataFrame(new_rows)

# total added per zip (keep as Series)
added_total_zip = pd.Series(0.0, index=pd.Index(Z, name="zipcode"))
if not exp_df.empty:
    added_total_zip = added_total_zip.add(exp_df.groupby("zipcode")["expansion_slots"].sum(), fill_value=0.0)
if not new_df.empty:
    added_total_zip = added_total_zip.add(new_df.groupby("zipcode")["added_slots_total"].sum(), fill_value=0.0)

# total 0–5 added per zip
added_05_zip = pd.Series(0.0, index=pd.Index(Z, name="zipcode"))
if not exp_df.empty:
    added_05_zip = added_05_zip.add(exp_df.groupby("zipcode")["expansion_0_5"].sum(), fill_value=0.0)
if not new_df.empty:
    added_05_zip = added_05_zip.add(new_df.groupby("zipcode")["added_slots_0_5_designated"].sum(), fill_value=0.0)

# coverage check — C1 & C2
rows = []
for z in Z:
    target12  = float(Target_slots.get(z, 0.0))
    current12 = float(Cnow_tot.get(z, 0.0))
    added12   = float(added_total_zip.get(z, 0.0))

    target05  = float(tau_05 * N0_5_series.loc[z])
    current05 = float(Cnow_05_series.loc[z])
    added05   = float(added_05_zip.get(z, 0.0))

    rows.append({
        "zipcode": z,
        "current_slots_0_12": current12,
        "added_slots_0_12": added12,
        "target_slots_0_12": target12,
        "meets_C1_total": (current12 + added12 + 1e-6) >= target12,
        "current_slots_0_5": current05,
        "added_slots_0_5": added05,
        "target_slots_0_5": target05,
        "meets_C2_0_5": (current05 + added05 + 1e-6) >= target05,
    })
cover_df = pd.DataFrame(rows)

# =========================================
# Step 9 — write CSVs + print summary stats
# =========================================
exp_out  = os.path.join(OUTPUT_PATH, "p1_sol_expansions.csv")
new_out  = os.path.join(OUTPUT_PATH, "p1_sol_newbuild.csv")
cov_out  = os.path.join(OUTPUT_PATH, "p1_sol_zip_coverage_check.csv")

if not exp_df.empty:
    exp_df.to_csv(exp_out, index=False)
if not new_df.empty:
    new_df.to_csv(new_out, index=False)
cover_df.to_csv(cov_out, index=False)

print("\n=== P1 Solution Summary ===")
print("Status:", status_txt)
print("Objective (total $):", f"{obj_val:,.0f}" if obj_val is not None else None)
print("# facilities expanded:", 0 if exp_df.empty else len(exp_df))
print("# new facilities built:", 0 if new_df.empty else len(new_df))
print("ZIPs meeting C1 (0–12):", int(cover_df['meets_C1_total'].sum()), "/", len(cover_df))
print("ZIPs meeting C2 (0–5): ", int(cover_df['meets_C2_0_5'].sum()), "/", len(cover_df))

if obj_val is not None:
    exp_fixed_cost = float(exp_df['cost_expansion_fixed'].sum()) if 'cost_expansion_fixed' in exp_df.columns else 0.0
    build_cost = float(new_df['cost_build'].sum()) if not new_df.empty else 0.0
    equip_cost = (
        float(exp_df['cost_equip_0_5_exp'].sum()) if 'cost_equip_0_5_exp' in exp_df.columns else 0.0
    ) + (
        float(new_df['cost_equip_0_5_new'].sum()) if 'cost_equip_0_5_new' in new_df.columns else 0.0
    )
    print(f"Cost breakdown — build: ${build_cost:,.0f}, expansion_fixed: ${exp_fixed_cost:,.0f}, equipment: ${equip_cost:,.0f}")

print("\nWrote:")
if not exp_df.empty: print(" -", exp_out)
if not new_df.empty: print(" -", new_out)
print(" -", cov_out)


Mounted at /content/drive
✓ Google Drive mounted.
DATA_PATH: /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/data/
OUTPUT_PATH: /content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/
✓ Loaded cleaned inputs. 
 master: (1375, 14) 
 fac   : (14435, 5) 
 locs  : (137500, 4) 
 zipage: (1604, 5)
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2726201
Academic license 2726201 - for non-commercial use only - registered to hd___@columbia.edu
✓ Gurobi WLS environment created.
ZIPs with need — C1: 1229, C2: 1237, union: 1299; kept sites: 129900
Sites after per-ZIP cap (20): 25980
C1 impossible ZIPs: 1 ['11219']
C2 impossible ZIPs: 3 ['10950', '11219', '11230']
Model size: 199185 vars, 164121 rows
Set parameter MIPGap to value 0.001
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set 

# Part 2

In [5]:
"""
Part 2 - BATCHED SOLUTION WITH RETRY LOGIC
Solves zipcodes in batches, then retries failed batches in smaller chunks
v4: custom batch sizes
"""

import math
import pandas as pd
import numpy as np
from itertools import combinations
import os
import gurobipy as gp
from gurobipy import GRB

# WLS license
params = {
    "WLSACCESSID": "b92cfff8-0279-4652-9cb7-3e6bcdb967cd",
    "WLSSECRET":   "e6220bd1-2657-40f8-ba01-6cebcb97db7f",
    "LICENSEID":   2726201,
}
env = gp.Env(params=params)

# ========== PARAMETERS ==========
MIN_DISTANCE_MILES = 0.06
EXPANSION_CAP_FRAC = 0.40
LOCATIONS_PER_ZIP_MAX = 50
MAX_ZIPS_TO_CONSIDER = 1400
LOCATIONS_PER_ZIP_MIN = 3

TIME_LIMIT_SECONDS = 600
MAX_FACILITIES_PER_ZIP = 30

SEGMENT_BREAKPOINTS = [0.10, 0.15, 0.20, 0.30, 0.40]
SEGMENT_COSTS = [200, 400, 1000, 2000, 3000]
BASE_COST = 20000

NEW_FACILITY_SIZES = {
    'small': {'max_slots_0_5': 80, 'cost': 65000, 'max_slots_allages': 100},
    'medium': {'max_slots_0_5': 160, 'cost': 95000, 'max_slots_allages': 200},
    'large': {'max_slots_0_5': 320, 'cost': 115000, 'max_slots_allages': 400}
}

EQUIP_COST_PER_SLOT_0_5 = 100

def haversine_miles(lat1, lon1, lat2, lon2):
    R = 3958.8
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

def piecewise_expansion_cost(nf, x):
    if nf == 0 or x <= 0:
        return 0.0

    frac = x / nf
    cumulative_cost = 0
    remaining_x = x

    for i in range(len(SEGMENT_BREAKPOINTS)):
        if frac <= SEGMENT_BREAKPOINTS[i]:
            marginal_cost = BASE_COST / nf + SEGMENT_COSTS[i]
            cumulative_cost += marginal_cost * remaining_x
            return cumulative_cost
        else:
            if i == 0:
                segment_slots = int(nf * SEGMENT_BREAKPOINTS[i])
            else:
                segment_slots = int(nf * SEGMENT_BREAKPOINTS[i]) - int(nf * SEGMENT_BREAKPOINTS[i-1])

            marginal_cost = BASE_COST / nf + SEGMENT_COSTS[i]
            cumulative_cost += marginal_cost * segment_slots
            remaining_x -= segment_slots

    return 1e9

def solve_batch(batch_zips, batch_name, all_expansions, all_newbuilds, all_verification, fac_index, pl_index):
    """Solve a single batch of zipcodes"""

    # Select locations for this batch
    selected_locations = []
    selected_ids = set()

    for _, zip_row in batch_zips.iterrows():
        zipcode = str(zip_row['zipcode'])
        n_to_select = int(zip_row['min_locations_needed'])

        candidates = locs[locs['zipcode'] == zipcode].copy()

        if candidates.empty:
            continue

        existing_in_zip = fac[fac['zipcode'] == zipcode]

        # PRE-FILTER
        valid_candidates = []
        for _, loc_row in candidates.iterrows():
            loc_lat = loc_row['latitude']
            loc_lon = loc_row['longitude']

            too_close = False
            for _, fac_row in existing_in_zip.iterrows():
                dist = haversine_miles(loc_lat, loc_lon, fac_row['latitude'], fac_row['longitude'])
                if dist < MIN_DISTANCE_MILES:
                    too_close = True
                    break

            if not too_close:
                valid_candidates.append(loc_row)

        candidates = pd.DataFrame(valid_candidates)

        if not candidates.empty:
            if not existing_in_zip.empty:
                mean_lat = existing_in_zip['latitude'].mean()
                mean_lon = existing_in_zip['longitude'].mean()

                candidates['dist_to_center'] = candidates.apply(
                    lambda row: haversine_miles(row['latitude'], row['longitude'], mean_lat, mean_lon),
                    axis=1
                )
                candidates = candidates.sort_values('dist_to_center', ascending=False)

            selected_count = 0
            for _, loc_row in candidates.iterrows():
                if selected_count >= n_to_select:
                    break

                loc_id = loc_row['location_id']
                if loc_id not in selected_ids:
                    loc_lat = loc_row['latitude']
                    loc_lon = loc_row['longitude']

                    too_close_to_selected = False
                    for already_selected in selected_locations:
                        if str(already_selected['zipcode']) == zipcode:
                            dist = haversine_miles(loc_lat, loc_lon,
                                                 already_selected['latitude'],
                                                 already_selected['longitude'])
                            if dist < MIN_DISTANCE_MILES:
                                too_close_to_selected = True
                                break

                    if not too_close_to_selected:
                        selected_locations.append(loc_row)
                        selected_ids.add(loc_id)
                        selected_count += 1

    selected_df = pd.DataFrame(selected_locations)

    if selected_df.empty:
        print(f"  No locations selected for {batch_name}, skipping...")
        return False

    print(f"  Selected {len(selected_df)} locations")

    # Setup optimization
    selected_zipcodes = set(selected_df['zipcode'].astype(str))
    selected_df['location_id'] = selected_df['location_id'].astype(str)
    pl_index = selected_df.set_index('location_id', drop=False) # Update pl_index for this batch
    selected_locs_list = list(pl_index.index)

    facilities_stage2 = []
    for z in selected_zipcodes:
        zip_facilities = fac[fac['zipcode'] == z].copy()

        if zip_facilities.empty:
            continue

        zip_facilities = zip_facilities.sort_values('capacity', ascending=False)

        for _, row in zip_facilities.head(MAX_FACILITIES_PER_ZIP).iterrows():
            facility_id = str(row['facility_id'])
            try:
                current_cap = int(row['capacity']) if 'capacity' in row else 0
                if current_cap > 0:
                    facilities_stage2.append(facility_id)
            except:
                pass

    print(f"  Problem size: {len(facilities_stage2)} facilities, {len(selected_locs_list)} locations")

    # fac_index = fac.set_index('facility_id', drop=False) # Already done outside the function
    zipdf_index = master.set_index('zipcode', drop=False)

    current_capacity = {}
    for f in facilities_stage2:
        try:
            if str(f).isdigit() and int(f) in fac_index.index:
                nf = int(fac_index.at[int(f), 'capacity'])
            elif f in fac_index.index:
                nf = int(fac_index.at[f, 'capacity'])
            else:
                nf = 0
        except:
            nf = 0
        current_capacity[f] = nf

    # Build model
    m = gp.Model(batch_name, env=env)
    m.setParam('TimeLimit', TIME_LIMIT_SECONDS)
    m.setParam('OutputFlag', 0)

    x = {}
    for f in facilities_stage2:
        nf = current_capacity[f]
        max_exp = int(nf * EXPANSION_CAP_FRAC) if nf > 0 else 0
        x[f] = m.addVar(lb=0, ub=max_exp, vtype=GRB.INTEGER, name=f"x_{f}")

    y = {}
    for j in selected_locs_list:
        for s in NEW_FACILITY_SIZES.keys():
            y[j,s] = m.addVar(vtype=GRB.BINARY, name=f"y_{j}_{s}")

    anybuild = {}
    for j in selected_locs_list:
        anybuild[j] = m.addVar(vtype=GRB.BINARY, name=f"anybuild_{j}")

    m.update()

    # Piecewise expansion costs
    for f in facilities_stage2:
        nf = current_capacity[f]
        if nf == 0:
            continue

        max_exp_slots = int(nf * EXPANSION_CAP_FRAC)

        points = [0]
        values = [0]

        for i, bp in enumerate(SEGMENT_BREAKPOINTS):
            if bp <= EXPANSION_CAP_FRAC:
                bp_slots = int(nf * bp)
                if bp_slots > points[-1] and bp_slots <= max_exp_slots:
                    cum_cost = 0
                    for j in range(i + 1):
                        if j == 0:
                            seg_start = 0
                        else:
                            seg_start = int(nf * SEGMENT_BREAKPOINTS[j-1])

                        seg_end = min(int(nf * SEGMENT_BREAKPOINTS[j]), bp_slots)

                        if seg_end > seg_start:
                            marginal = BASE_COST/nf + SEGMENT_COSTS[j]
                            cum_cost += marginal * (seg_end - seg_start)

                    points.append(bp_slots)
                    values.append(cum_cost)

        if points[-1] < max_exp_slots:
            seg_idx = len(SEGMENT_COSTS) - 1
            cum_cost = values[-1]
            marginal = BASE_COST/nf + SEGMENT_COSTS[seg_idx]
            cum_cost += marginal * (max_exp_slots - points[-1])
            points.append(max_exp_slots)
            values.append(cum_cost)

        if len(points) > 1:
            m.setPWLObj(x[f], points, values)

    m.update()

    # Constraints
    for j in selected_locs_list:
        m.addConstr(gp.quicksum(y[j,s] for s in NEW_FACILITY_SIZES.keys()) <= 1, name=f"one_size_{j}")

    for j in selected_locs_list:
        m.addConstr(anybuild[j] == gp.quicksum(y[j,s] for s in NEW_FACILITY_SIZES.keys()), name=f"link_{j}")

    # Distance constraints
    for z in selected_zipcodes:
        existing_in_z = [f for f in facilities_stage2
                         if str(f).isdigit() and int(f) in fac_index.index
                         and str(fac_index.at[int(f), 'zipcode']) == z]
        new_in_z = [j for j in selected_locs_list if str(pl_index.at[j, 'zipcode']) == z]

        for f in existing_in_z:
            try:
                lat_f = float(fac_index.at[int(f), 'latitude'])
                lon_f = float(fac_index.at[int(f), 'longitude'])
            except:
                continue

            for j in new_in_z:
                try:
                    lat_j = float(pl_index.at[j, 'latitude'])
                    lon_j = float(pl_index.at[j, 'longitude'])
                except:
                    continue

                dist = haversine_miles(lat_f, lon_f, lat_j, lon_j)
                if dist < MIN_DISTANCE_MILES:
                    m.addConstr(anybuild[j] == 0, name=f"dist_{f}_{j}")

        for j1, j2 in combinations(new_in_z, 2):
            try:
                lat1 = float(pl_index.at[j1, 'latitude'])
                lon1 = float(pl_index.at[j1, 'longitude'])
                lat2 = float(pl_index.at[j2, 'latitude'])
                lon2 = float(pl_index.at[j2, 'longitude'])

                dist = haversine_miles(lat1, lon1, lat2, lon2)
                if dist < MIN_DISTANCE_MILES:
                    m.addConstr(anybuild[j1] + anybuild[j2] <= 1, name=f"dist_{j1}_{j2}")
            except:
                continue


    # Coverage constraints
    for z in selected_zipcodes:
        if z not in zipdf_index.index:
            continue

        desert_threshold = float(zipdf_index.at[z, 'desert_threshold'])
        children_0_5 = float(zipdf_index.at[z, 'pop_0_5'])

        existing_in_z = [f for f in facilities_stage2
                         if str(f).isdigit() and int(f) in fac_index.index
                         and str(fac_index.at[int(f), 'zipcode']) == z]

        current_total = sum(current_capacity.get(f, 0) for f in existing_in_z)
        expansion_slots = gp.quicksum(x[f] for f in existing_in_z)

        new_in_z = [j for j in selected_locs_list if str(pl_index.at[j, 'zipcode']) == z]
        new_slots_total = gp.quicksum(
            NEW_FACILITY_SIZES[s]['max_slots_allages'] * y[j,s]
            for j in new_in_z for s in NEW_FACILITY_SIZES.keys()
        )
        new_slots_0_5 = gp.quicksum(
            NEW_FACILITY_SIZES[s]['max_slots_0_5'] * y[j,s]
            for j in new_in_z for s in NEW_FACILITY_SIZES.keys()
        )

        total_slots = current_total + expansion_slots + new_slots_total

        m.addConstr(total_slots >= desert_threshold, name=f"desert_{z}")

        min_slots_0_5 = children_0_5 * (2.0/3.0)
        current_0_5_approx = current_total * 0.8 # Approximation based on overall facility data
        total_0_5_capacity = current_0_5_approx + expansion_slots * 0.8 + new_slots_0_5 # Approximation
        m.addConstr(total_0_5_capacity >= min_slots_0_5, name=f"min_0_5_{z}")


    # Objective
    new_build_cost = gp.quicksum(
        NEW_FACILITY_SIZES[s]['cost'] * y[j,s]
        for j in selected_locs_list for s in NEW_FACILITY_SIZES.keys()
    )

    new_equip_cost = gp.quicksum(
        EQUIP_COST_PER_SLOT_0_5 * NEW_FACILITY_SIZES[s]['max_slots_0_5'] * y[j,s]
        for j in selected_locs_list for s in NEW_FACILITY_SIZES.keys()
    )

    m.setObjective(new_build_cost + new_equip_cost, GRB.MINIMIZE)

    # Solve
    print(f"  Solving {batch_name}...")
    m.optimize()

    if m.status == GRB.INFEASIBLE:
        print(f"  ❌ {batch_name} INFEASIBLE")
        return False
    elif m.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        print(f"  ✓ {batch_name}: ${m.objVal:,.2f}")

        # Extract results
        for f in facilities_stage2:
            val = int(round(x[f].X)) if x[f].X is not None else 0
            if val > 0:
                nf = current_capacity[f]
                cost = piecewise_expansion_cost(nf, val)
                # Get lat/lon for expansion
                fac_lat = float(fac_index.at[int(f), 'latitude']) if int(f) in fac_index.index and 'latitude' in fac_index.columns else None
                fac_lon = float(fac_index.at[int(f), 'longitude']) if int(f) in fac_index.index and 'longitude' in fac_index.columns else None

                all_expansions.append({
                    'batch': batch_name,
                    'facility_id': f,
                    'zipcode': str(fac_index.at[int(f), 'zipcode']) if int(f) in fac_index.index else None,
                    'latitude': fac_lat,
                    'longitude': fac_lon,
                    'added_slots': val,
                    'cost': float(cost)
                })

        for j in selected_locs_list:
            for s in NEW_FACILITY_SIZES.keys():
                if y[j,s].X is not None and y[j,s].X > 0.5:
                    meta = NEW_FACILITY_SIZES[s]
                    # Get lat/lon for new build
                    loc_lat = float(pl_index.at[j, 'latitude']) if j in pl_index.index and 'latitude' in pl_index.columns else None
                    loc_lon = float(pl_index.at[j, 'longitude']) if j in pl_index.index and 'longitude' in pl_index.columns else None

                    all_newbuilds.append({
                        'batch': batch_name,
                        'location_id': j,
                        'zipcode': str(pl_index.at[j, 'zipcode']) if j in pl_index.index else None,
                        'latitude': loc_lat,
                        'longitude': loc_lon,
                        'size': s,
                        'added_slots': meta['max_slots_allages'],
                        'build_cost': float(meta['cost']),
                        'equip_cost_0_5': float(EQUIP_COST_PER_SLOT_0_5 * meta['max_slots_0_5'])
                    })

        for z in selected_zipcodes:
            if z not in zipdf_index.index:
                continue

            threshold = float(zipdf_index.at[z, 'desert_threshold'])
            current_existing = float(zipdf_index.at[z, 'total_existing_slots'])

            existing_in_z = [f for f in facilities_stage2
                         if str(f).isdigit() and int(f) in fac_index.index
                         and str(fac_index.at[int(f), 'zipcode']) == z]
            added_exp = sum(int(round(x[f].X)) for f in existing_in_z if x[f].X is not None)

            new_in_z = [j for j in selected_locs_list if str(pl_index.at[j, 'zipcode']) == z]
            added_new = sum(
                NEW_FACILITY_SIZES[s]['max_slots_allages']
                for j in new_in_z for s in NEW_FACILITY_SIZES.keys()
                if y[j,s].X is not None and y[j,s].X > 0.5
            )

            total = current_existing + added_exp + added_new
            meets = total >= threshold - 1e-6

            all_verification.append({
                'batch': batch_name,
                'zipcode': z,
                'threshold': threshold,
                'current': current_existing,
                'added_exp': added_exp,
                'added_new': added_new,
                'total': total,
                'meets': meets
            })

        return True

    return False

print("Loading data...")

try:
    master = pd.read_csv(os.path.join(OUTPUT_PATH, 'zipcode_master.csv'), dtype={'zipcode': str})
    fac = pd.read_csv(os.path.join(OUTPUT_PATH, 'facilities_clean.csv'))
    locs = pd.read_csv(os.path.join(OUTPUT_PATH, 'locations_clean.csv'))
except FileNotFoundError as e:
    print(f"Error: {e}")
    exit(1)

for df in [master, fac, locs]:
    if 'zipcode' in df.columns:
        df['zipcode'] = df['zipcode'].astype(str).str.zfill(5)

# Create index DataFrames for faster lookups
fac_index = fac.set_index('facility_id')
locs['location_id'] = locs['location_id'].astype(str) # Ensure location_id is string for index
pl_index = locs.set_index('location_id')


print(f"Data loaded: {len(master)} zips, {len(fac)} facilities, {len(locs)} locations")

# ========== STAGE 1: IDENTIFY ALL NEEDY ZIPCODES ==========
print("\n" + "="*60)
print("STAGE 1: Identify needy zipcodes")
print("="*60)

master['need'] = master['desert_threshold'] - master['total_existing_slots']
master['need'] = master['need'].clip(lower=0)

print("\nCalculating expansion potential per zipcode...")
master['expansion_potential'] = 0.0

for idx, row in master.iterrows():
    zipcode = str(row['zipcode'])
    zip_facs = fac[fac['zipcode'] == zipcode].nlargest(MAX_FACILITIES_PER_ZIP, 'capacity')

    if not zip_facs.empty:
        total_current = zip_facs['capacity'].sum()
        max_expansion = int(total_current * EXPANSION_CAP_FRAC)
        master.at[idx, 'expansion_potential'] = max_expansion

master['need_after_expansion'] = (master['need'] - master['expansion_potential']).clip(lower=0)

LARGE_FACILITY_CAPACITY = 400
LARGE_FACILITY_0_5_CAPACITY = 320

master['locations_for_0_12'] = np.ceil(master['need_after_expansion'] / LARGE_FACILITY_CAPACITY).astype(int)

master['required_0_5'] = master['pop_0_5'] * (2.0/3.0)
master['have_0_5_current'] = master['total_existing_slots'] * 0.8 # Approximation
master['have_0_5_expansion'] = master['expansion_potential'] * 0.8 # Approximation
master['need_0_5_new'] = (master['required_0_5'] - master['have_0_5_current'] - master['have_0_5_expansion']).clip(lower=0)
master['locations_for_0_5'] = np.ceil(master['need_0_5_new'] / LARGE_FACILITY_0_5_CAPACITY).astype(int)

master['min_locations_needed'] = master[['locations_for_0_12', 'locations_for_0_5']].max(axis=1)
master['min_locations_needed'] = master['min_locations_needed'].astype(int)  # NO BUFFER
master['min_locations_needed'] = master['min_locations_needed'].clip(lower=LOCATIONS_PER_ZIP_MIN, upper=LOCATIONS_PER_ZIP_MAX)

need_sorted = master.sort_values('need_after_expansion', ascending=False)
top_zips = need_sorted.head(MAX_ZIPS_TO_CONSIDER)
top_zips = top_zips[top_zips['need_after_expansion'] > 0].copy()

if top_zips.empty:
    print("ERROR: No zipcodes with unmet need!")
    exit(1)

print(f"\nTotal needy zipcodes: {len(top_zips)}")

# Split high-need vs low-need
high_need = top_zips.head(500)
low_need = top_zips.iloc[500:]

# Small batches for high-need areas
high_batches = [high_need.iloc[i:i+25] for i in range(0, len(high_need), 25)]

# Normal batches for low-need areas
low_batches = [low_need.iloc[i:i+100] for i in range(0, len(low_need), 100)]

batches = high_batches + low_batches

print(f"High-need batches: {len(high_batches)} (25 zips each)")
print(f"Low-need batches: {len(low_batches)} (100 zips each)")
print(f"Total batches: {len(batches)}")

# ========== MAIN BATCH PROCESSING ==========
all_expansions = []
all_newbuilds = []
all_verification = []
failed_batches = []

for batch_idx, batch_zips in enumerate(batches):
    print(f"\n{'='*60}")
    print(f"BATCH {batch_idx+1}/{len(batches)}: {len(batch_zips)} zipcodes")
    print(f"{'='*60}")

    success = solve_batch(batch_zips, f"batch_{batch_idx+1}", all_expansions, all_newbuilds, all_verification, fac_index, pl_index)

    if not success:
        failed_batches.append((batch_idx, batch_zips))

# ========== RETRY FAILED BATCHES ==========
if failed_batches:
    print(f"\n{'='*60}")
    print(f"RETRYING {len(failed_batches)} FAILED BATCHES (10 zips per chunk)")
    print(f"{'='*60}")

    for batch_idx, failed_batch in failed_batches:
        print(f"\nRetrying batch {batch_idx+1}...")

        # Split into 10-zipcode chunks
        mini_batches = [failed_batch.iloc[i:i+10] for i in range(0, len(failed_batch), 10)]

        for mini_idx, mini_batch in enumerate(mini_batches):
            print(f"\n--- Chunk {mini_idx+1}/{len(mini_batches)}: {len(mini_batch)} zipcodes ---")

            success = solve_batch(mini_batch, f"retry_batch_{batch_idx+1}_chunk_{mini_idx+1}",
                                all_expansions, all_newbuilds, all_verification, fac_index, pl_index)

            if not success:
                print(f"  Still infeasible: {list(mini_batch['zipcode'])}")


# ========== SAVE RESULTS ==========
print(f"\n{'='*60}")
print("FINAL RESULTS")
print(f"{'='*60}")

exp_df = pd.DataFrame(all_expansions)
new_df = pd.DataFrame(all_newbuilds)
ver_df = pd.DataFrame(all_verification)

if not exp_df.empty:
    exp_df.to_csv(os.path.join(OUTPUT_PATH, 'part2_expansions_final.csv'), index=False)
    print(f"Expansions: {len(exp_df)}")

if not new_df.empty:
    new_df.to_csv(os.path.join(OUTPUT_PATH, 'part2_newbuilds_final.csv'), index=False)
    print(f"New builds: {len(new_df)}")

if not ver_df.empty:
    ver_df.to_csv(os.path.join(OUTPUT_PATH, 'part2_verification_final.csv'), index=False)
    met_count = ver_df['meets'].sum()
    total_count = len(ver_df)
    print(f"Zipcodes meeting threshold: {met_count}/{total_count}")

    if not exp_df.empty or not new_df.empty:
        total_cost = 0
        if not exp_df.empty:
            total_cost += exp_df['cost'].sum()
        if not new_df.empty:
            total_cost += new_df['build_cost'].sum() + new_df['equip_cost_0_5'].sum()
        print(f"Total cost: ${total_cost:,.2f}")

    # Show unsolved zipcodes
    if met_count < len(top_zips):
        solved_zips = set(ver_df['zipcode'])
        unsolved_zips = [z for z in top_zips['zipcode'] if z not in solved_zips]
        print(f"Unsolved zipcodes: {len(unsolved_zips)}")
        if len(unsolved_zips) <= 20:
            print(f"Unsolved: {unsolved_zips}")


print(f"\n{'='*60}")
print("DONE")
print(f"{'='*60}")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2726201
Academic license 2726201 - for non-commercial use only - registered to hd___@columbia.edu
Loading data...
Data loaded: 1375 zips, 14435 facilities, 137500 locations

STAGE 1: Identify needy zipcodes

Calculating expansion potential per zipcode...

Total needy zipcodes: 1087
High-need batches: 20 (25 zips each)
Low-need batches: 6 (100 zips each)
Total batches: 26

BATCH 1/26: 25 zipcodes
  Selected 277 locations
  Problem size: 697 facilities, 277 locations
Set parameter TimeLimit to value 600
  Solving batch_1...
  ❌ batch_1 INFEASIBLE

BATCH 2/26: 25 zipcodes
  Selected 133 locations
  Problem size: 633 facilities, 133 locations
Set parameter TimeLimit to value 600
  Solving batch_2...
  ❌ batch_2 INFEASIBLE

BATCH 3/26: 25 zipcodes
  Selected 102 locations
  Problem size: 513 facilities, 102 locations
Set parameter TimeLimit to value 600
  Solving batch_3...
  ❌ batch_3 INFEASIBLE

BATCH 4/26:

In [6]:
"""
Diagnostic tool to understand why certain zipcodes cause infeasibility
"""

import pandas as pd
import os

# Assuming OUTPUT_PATH is defined
# OUTPUT_PATH = '/content/drive/MyDrive/IEORE4004 Optimization Models and Methods/IEOR4404_Proj1/cleaned_data/'

try:
    master = pd.read_csv(os.path.join(OUTPUT_PATH, 'zipcode_master.csv'), dtype={'zipcode': str})
    fac = pd.read_csv(os.path.join(OUTPUT_PATH, 'facilities_clean.csv'))
    locs = pd.read_csv(os.path.join(OUTPUT_PATH, 'locations_clean.csv'))
except Exception as e:
    print(f"Error loading data: {e}")
    exit(1)

# Normalize zipcode formats
for df in [master, fac, locs]:
    if 'zipcode' in df.columns:
        df['zipcode'] = df['zipcode'].astype(str).str.zfill(5)

# Parameters
MAX_FACILITIES_PER_ZIP = 20
EXPANSION_CAP_FRAC = 0.20

print("="*70)
print("INFEASIBILITY DIAGNOSTIC")
print("="*70)

# Analyze the problematic zipcode(s)
problem_zips = ['10952']  # From IIS output

for z in problem_zips:
    print(f"\n{'='*70}")
    print(f"Analyzing Zipcode: {z}")
    print(f"{'='*70}")

    if z not in master['zipcode'].values:
        print(f"  ⚠ Zipcode {z} not found in master data")
        continue

    zip_data = master[master['zipcode'] == z].iloc[0]

    # Population and thresholds
    pop_0_12 = float(zip_data['pop_0_12'])
    pop_0_5 = float(zip_data['pop_0_5'])
    desert_threshold = float(zip_data['desert_threshold'])
    current_slots = float(zip_data['total_existing_slots'])
    need = desert_threshold - current_slots

    print(f"\n📊 DEMAND:")
    print(f"  Population 0-12: {pop_0_12:.0f}")
    print(f"  Population 0-5: {pop_0_5:.0f}")
    print(f"  Desert threshold (required): {desert_threshold:.0f}")
    print(f"  Current slots: {current_slots:.0f}")
    print(f"  Unmet need: {need:.0f}")

    # Existing facilities
    existing_facs = fac[fac['zipcode'] == z].copy()
    print(f"\n🏢 EXISTING FACILITIES:")
    print(f"  Total facilities in zip: {len(existing_facs)}")

    if len(existing_facs) > 0:
        existing_facs = existing_facs.sort_values('capacity', ascending=False)
        top_facs = existing_facs.head(MAX_FACILITIES_PER_ZIP)

        print(f"  Top {MAX_FACILITIES_PER_ZIP} facilities (by capacity):")
        total_current_capacity = 0
        total_max_expansion = 0

        for idx, row in top_facs.iterrows():
            cap = int(row['capacity'])
            max_exp = int(cap * EXPANSION_CAP_FRAC)
            total_current_capacity += cap
            total_max_expansion += max_exp
            print(f"    Facility {row['facility_id']}: capacity={cap}, max_expansion={max_exp}")

        print(f"\n  Summary of top {len(top_facs)} facilities:")
        print(f"    Current capacity: {total_current_capacity}")
        print(f"    Max expansion (20%): {total_max_expansion}")
        print(f"    Max total after expansion: {total_current_capacity + total_max_expansion}")
    else:
        print(f"  ⚠ NO existing facilities in this zipcode!")
        total_current_capacity = 0
        total_max_expansion = 0

    # Potential new locations
    potential_locs = locs[locs['zipcode'] == z]
    print(f"\n📍 POTENTIAL NEW LOCATIONS:")
    print(f"  Available locations: {len(potential_locs)}")

    # Maximum possible from new builds (assuming all large facilities)
    max_new_capacity = len(potential_locs) * 400  # 400 = max slots for large facility
    print(f"  Max capacity if all locations built as 'large': {max_new_capacity}")

    # Supply vs Demand Analysis
    print(f"\n⚖️ SUPPLY vs DEMAND:")
    print(f"  Current capacity: {total_current_capacity}")
    print(f"  + Max expansion: {total_max_expansion}")
    print(f"  + Max new builds: {max_new_capacity}")
    print(f"  = Max total supply: {total_current_capacity + total_max_expansion + max_new_capacity}")
    print(f"  Required (threshold): {desert_threshold}")

    gap = (total_current_capacity + total_max_expansion + max_new_capacity) - desert_threshold

    if gap >= 0:
        print(f"  ✓ SUFFICIENT: Surplus of {gap:.0f} slots")
    else:
        print(f"  ❌ INSUFFICIENT: Deficit of {abs(gap):.0f} slots")
        print(f"\n  🔍 ROOT CAUSE:")
        if len(existing_facs) == 0:
            print(f"    - No existing facilities to expand")
        if len(potential_locs) == 0:
            print(f"    - No potential locations for new builds")
        if gap < 0:
            print(f"    - Even with maximum expansion and all new builds, cannot meet threshold")

    # 0-5 constraint check
    min_slots_0_5 = pop_0_5 * (2.0/3.0)
    # Approximate: 50% of existing + expansions serve 0-5, new builds have explicit 0-5 capacity
    # The variables expansion_slots and new_slots_0_5 are defined in the previous cell (qn_0-PkPLfwe)
    # Since this is a diagnostic tool, we can approximate or assume these values based on max potential
    # For a more accurate check, these would need to be calculated within this cell or passed as parameters.
    # For now, we'll use maximum potential values for a simplified check.
    expansion_0_5_approx = total_max_expansion * 0.8 # Assuming 80% of expanded slots are for 0-5
    max_new_0_5 = len(potential_locs) * 320 # Max 0-5 slots for large facility is 320

    total_0_5_max = (total_current_capacity * 0.8) + expansion_0_5_approx + max_new_0_5 # Assuming 80% of current capacity is for 0-5

    print(f"\n👶 0-5 AGE GROUP CONSTRAINT:")
    print(f"  Required (2/3 of pop_0_5): {min_slots_0_5:.0f}")
    print(f"  Max possible 0-5 slots: {total_0_5_max:.0f}")

    gap_0_5 = total_0_5_max - min_slots_0_5
    if gap_0_5 >= 0:
        print(f"  ✓ SUFFICIENT: Surplus of {gap_0_5:.0f} slots")
    else:
        print(f"  ❌ INSUFFICIENT: Deficit of {abs(gap_0_5):.0f} slots")

print(f"\n{'='*70}")
print("RECOMMENDATIONS:")
print(f"{'='*70}")

for z in problem_zips:
    if z not in master['zipcode'].values:
        continue

    zip_data = master[master['zipcode'] == z].iloc[0]
    desert_threshold = float(zip_data['desert_threshold'])
    current_slots = float(zip_data['total_existing_slots'])

    existing_facs = fac[fac['zipcode'] == z]
    potential_locs = locs[locs['zipcode'] == z]

    top_facs = existing_facs.sort_values('capacity', ascending=False).head(MAX_FACILITIES_PER_ZIP)
    total_current = top_facs['capacity'].sum() if len(top_facs) > 0 else 0
    max_expansion = int(total_current * EXPANSION_CAP_FRAC)
    max_new = len(potential_locs) * 400

    max_total = total_current + max_expansion + max_new

    if max_total < desert_threshold:
        print(f"\nZipcode {z}:")
        print(f"  1. ❌ INFEASIBLE - Cannot meet threshold even with all options")
        print(f"     Solutions:")
        print(f"       a) Increase EXPANSION_CAP_FRAC from 0.20 to 0.30 or higher")
        print(f"       b) Include facilities from neighboring zipcodes (relaxed distance)")
        print(f"       c) Consider this zipcode acceptable as 'high-need desert' in reporting")
        print(f"       d) Add more potential locations to this zipcode")
    else:
        print(f"\nZipcode {z}:")
        print(f"  ✓ Feasible in theory, but may have other issues:")
        print(f"    - Check distance constraints (MIN_DISTANCE_MILES)")
        print(f"    - Check 0-5 capacity requirements")
        print(f"    - Increase MAX_FACILITIES_PER_ZIP to include more facilities")

print(f"\n{'='*70}")

INFEASIBILITY DIAGNOSTIC

Analyzing Zipcode: 10952

📊 DEMAND:
  Population 0-12: 16082
  Population 0-5: 7131
  Desert threshold (required): 5307
  Current slots: 581
  Unmet need: 4726

🏢 EXISTING FACILITIES:
  Total facilities in zip: 31
  Top 20 facilities (by capacity):
    Facility 533013: capacity=123, max_expansion=24
    Facility 848103: capacity=16, max_expansion=3
    Facility 905995: capacity=16, max_expansion=3
    Facility 746053: capacity=16, max_expansion=3
    Facility 848300: capacity=16, max_expansion=3
    Facility 467838: capacity=16, max_expansion=3
    Facility 148121: capacity=16, max_expansion=3
    Facility 847334: capacity=16, max_expansion=3
    Facility 847612: capacity=16, max_expansion=3
    Facility 849148: capacity=16, max_expansion=3
    Facility 35664: capacity=16, max_expansion=3
    Facility 843007: capacity=16, max_expansion=3
    Facility 27131: capacity=16, max_expansion=3
    Facility 349674: capacity=16, max_expansion=3
    Facility 752596: capa

In [7]:
# params = {
# "WLSACCESSID": "b92cfff8-0279-4652-9cb7-3e6bcdb967cd",
# "WLSSECRET":"e6220bd1-2657-40f8-ba01-6cebcb97db7f",
# "LICENSEID":2726201,
# }
# env = gp.Env(params=params)
